# Pinecone

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
pip install -q pinecone-client

DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade -q pinecone-client

DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip show pinecone-client

Name: pinecone-client
Version: 3.1.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /home/nishihara/.local/lib/python3.11/site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pinecone import Pinecone
pc = Pinecone()
# pc = Pinecone(api_key='YOUR_API_KEY')
pc.list_indexes()

{'indexes': []}

## Working with Pinecone Indexes

In [8]:
pc.list_indexes()

{'indexes': []}

In [9]:
pc.describe_index('langchain')

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '1b9c138e8f71de7415b45ea0a493cc51', 'Date': 'Fri, 15 Mar 2024 21:23:44 GMT', 'Server': 'Google Frontend', 'Content-Length': '84', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource langchain not found"},"status":404}


In [10]:
pc.list_indexes().names()

[]

In [31]:
from pinecone import PodSpec
index_name = 'langchain'

if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    pc.create_index(
            name=index_name,
            dimension=1536, # recommended size
            metric='cosine',
            spec=PodSpec(
                environment='gcp-starter'
            )
        )
    print('Index created!')
else:
    print(f'Index {index_name} already exists!')

Creating index langchain
Index created!


In [30]:
index_name = 'langchain'
if index_name in pc.list_indexes().names():
    print(f'Deleting index {index_name}....')
    pc.delete_index(index_name)
    print('Done')

Deleting index langchain....


In [32]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Working with Vectors

In [33]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_name = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids,vectors))

{'upserted_count': 5}

In [23]:
# updating vectors
index.upsert(vectors=[('c', [0.5] * 1536)])

{'upserted_count': 1}

In [24]:
# fetch vectors
# index = pc.Index(index_name)
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'c': {'id': 'c',
                   'values': [0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
             

In [27]:
# deleting vectors
index.delete(ids=['b','c'])

{}

In [28]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 3e-05,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [29]:
# if we fetch not existing id, it does not give errors but returns empty vector
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [34]:
# query
query_vector = [random.random() for _ in range(1536)]

index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'c', 'score': 0.752731264, 'values': []},
             {'id': 'a', 'score': 0.751929402, 'values': []},
             {'id': 'b', 'score': 0.748736262, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

## Pinecone Namespaces

In [36]:
index.describe_index_stats()
index = pc.Index('langchain')

import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index.upsert(vectors=zip(ids,vectors))

{'upserted_count': 5}

In [37]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
# print(vectors)
ids = list('xyz')

index.upsert(vectors=zip(ids,vectors), namespace='first-namespace')

{'upserted_count': 3}

In [41]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(2)]
# print(vectors)
ids = list('xyz')

index.upsert(vectors=zip(ids,vectors), namespace='second-namespace')

{'upserted_count': 2}

In [42]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0001,
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 10}

In [44]:
index.fetch(ids=['x'],namespace='first-namespace')

{'namespace': 'first-namespace',
 'usage': {'read_units': 1},
 'vectors': {'x': {'id': 'x',
                   'values': [0.11089059,
                              0.636618197,
                              0.48768127,
                              0.935596585,
                              0.958088577,
                              0.798221648,
                              0.381115526,
                              0.773925602,
                              0.715135574,
                              0.8467924,
                              0.627556384,
                              0.0815004855,
                              0.96808207,
                              0.0447253212,
                              0.662280083,
                              0.431869686,
                              0.723523438,
                              0.0182384793,
                              0.0135900564,
                              0.554686904,
                              0.819282472,
      

In [45]:
index.delete(delete_all=True,namespace='first-namespace')

{}

In [46]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 7e-05,
 'namespaces': {'': {'vector_count': 5},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 7}